##**[DACON] 도메인 특화 평가 지표를 활용한 건설 사고 대응 시스템 개발**

### **주제 및 데이터 설명**

**대회 설명**
- 주제: 건설공사 안전사고 대응 및 재발 방지 대책 AI 모델 개발
- 설명: 건설공사 사고 상황 데이터를 바탕으로 사고 원인을 분석하고 재발방지 대책을 포함한 대응착을 자동으로 생성하는 AI 모델 개발

**데이터**

train.csv
- ID : 샘플별 고유 ID
- 발생일시
- 사고인지 시간
- 날씨
- 기온
- 습도
- 공사종류
- 연면적
- 층 정보
- 인적사고
- 물적사고
- 공종
- 사고객체
- 작업프로세스
- 장소
- 부위
- 사고원인
- 재발방지대책 및 향후조치계획 [예측 목표]


### **코드 흐름**

**EDA 및 Feature Engineering & preprocessing**

1. 예측 목표인 '재발방지대책 및 향후조치계획'의 상위 5개의 답변 분석

  -> 빈도 기준 상위 5개의 답변이 답변의 모호성, 표현의 반복성, 대응의 획일성 등으로 '안전교육 실시', '재발 방지 대책', '향후 조치 계획' 등의 유의미 하지 않은 문장을 확인
  => 이는 학습시 미세조정으로 활용하기 어렵고, 검증시 우수한 답변인지 평가하기 어려움
2. 프로젝트의 요구사항 확인
- LLM 기반 학습 데이터 구축 전략
- 도메인 특화 평가 지표 설계 등 건설 사고 대응 시스템 개발

3. Missing (결측치)
- Train.csv에서 사고 맥락의 이해에 중요한 인적사고, 공종, 사고객체, 작업프로세스, 사고 원인 변수에서 결측치가 존재 -> 제거
- Test.csv에서 임의를 샘플을 제거할 수 없음 -> 결측치를 빈 문자열로 대체
- 물적 사고 변수의 경우 '없음' 값의 빈도가 높음 -> 결측치를 없음으로 대체
 -> Train.csv는 총 23,421개의 샘플에서 22,576개의 샘플로 전처리됨 (4%의 학습 데이터 손실 발생)

**전반적인 모델 흐름**

1. Few-shot Prompting
- 몇개의 예제를 프롬프트에 포함하여 LLM의 추론 성능을 향상시키는 기법
- 코사인 유사도 행렬을 기반으로 train.csv로부터 관련성 있는 Top-5개의 Fwe-shot Prompting을 진행
- 유사도는 '사고원인', '공종', '작업프로세스', '장소', '사고객체', '부위'변수의 가중평균으로 산출
- 사고의 원인 뿐만아니라 사고가 발생한 환경에 대한 맥락도 고려하여 유사도 계산의 정밀도를 향상시킴

2. Prompt Engineering
- 설계 전략: System Message, User Message의 2단계로 구성

(1) System Message
- LLM에 건설 사고 분석 전문가의 역할을 부여하고 체계적이고 단계적인 분석을 진행하도록함
  - 주어진 사고 상황에 대한 단계적 분석을 통하여 주요 위험 요소와 원인을 파악
  - 검색된 유사 사고사례에 대한 각 사고 변수 항목을 종합적으로 비교분석
  - 사고원인과 사고 맥락을 기반으로 단계적으로 추록하여 결론 도출

(2) User Message
- 제공된 유사 사고사례를 통한 합리적인 추론 과정과 명확한 출력의 답변 생성을 목적으로 설계
  - 유사도 계산을 위해 train.csv에서 5가지 유사 사고 사례를 제시
  - 유사 사례와 사고 상황을 참고하여 구체적이고 종합적인 대책을 생성하도록 유도
  - 하나씩 차근차근 생각한뒤라는 Zero-shot CoT로 노이즈가 강한 데이터에서 모델의 추론 성능을 최대한 향상함
  - 대응대책으로 시작하는 출력형식을 명확히 지정하여 일관성있는 답변을 유도

3. Chain of Thought 기반 LLM
- 알리바바 클라우드의 오픈 소스 언어 모델 Qwen 시리즈의 추론 특화 LLM인 QwQ-32B를 사용
  - CoT기법과 강화 학습으로 최적화된 LLM으로 DeepSeek-R1, OpenAI-o1-mini등 state-of-art모델과 비교시 적은 파라미터로 우수한 사고 및 추론 능력을 보임
- 이를 기반으로 QLoRA를 활용한 4비트 양자화를 적용하여 단일 RTX3090환경에서도 서빙 가능성을 확인
- 추론 시 CoT기법의 <think> 태그를 추출하여 건설 공사 사고 상황을 분석하고 대응책 답변을 해석할 수 있음

4. LLM을 이용하기 위한 비교실험 정성적 평가
- QwQ-32B와 Phi-4, Gemma3 등의 성능을 직접적으로 비교함
  - Phi-4를 통하여 현재 구축된 학습데이터로 LLM을 미세 조정하면 답변의 품질이 떨어지는 것을 정성적으로 확인
  - Few-Shot Prompting을 통하여 사고 상황 정보 변수들울 참고하여 정보량이 풍부하고 유의미한 문장을 생성하는 것을 확인
  - QwQ가 Gemma-3에 비하여 해석 가능성 측면에서 강점을 가진 것을 확인

5.  LLM 기반 대응책 답변 생성
- 사고 상황정보를 입력하면 BackEnd에서 FAISS DB로부터 유사한 사고상황 사례를 검색하고, 이를 contest로 사용하여 프롬프트를 구성하고 LLM을 통해 2개의 대응책 답변을 생성함
- 이때 LLM을 실행할 수 있도록 도와주는 도구인 Oliama를 사용하여 시스템을 구축

6. 사용자 피드백 기반 데이터 구축
- LLM을 통해 생성된 2개의 대응책답변을 사용자에게 보여줄 수 있도록 FrontEnd로 전달
- 이를 사용자가 비교하고 더 나은 답변을 선택 가능, 이후 해당 답변을 확정하고 responses.json에 기록

7. 평가지표: 코사인 유사도 / 자카드 유사도
- 기존 평가 지표로예측 문장과 정답 문장 간의 코사인 유사도와 자카드 유사도의 가중평균을 이용함
  - 코사인 유사도: 의미론적 유사성을 측정
  - 자카드 유사도: 사용된 단어들의 집합을 비교해 어휘적 유사성 측정  
- 이러한 기존 평가 지표는 사고 원인에 대한 생성된 답변의 문맥을 반영하지 못하고 자연스러운 문장 생성하지 못하는 단점 발생
  -> 건설 공사 도메인에 특화된 추가 평가 지표를 설계 (Sentiment+Accident의 가중평균)
  - Sentiment: 사고 원인과 생성된 답변이 얼마나 의미론적으로 연관성이 있는지 평가하는 BERT Score기반의 지표
  - Accident: 생성된 답변에 사고상황 정보가 얼마나 포함되었는지 여부를 반영하는 평가지표
  => 사고 대응 연관성과 사고 맥락 반영도를 동시에 고려하면서 생성된 재발방지 대책 및 향후조치계획 품질을 정량적으로 판단 가능

8. 결론
- Gemma3은 Sentiment 점수에서 QwQ보다 좋은 성능을 보이나, Accident 점수가 낮은 성능을 보임

-> QwQ모델이 CoT 추론 과정을 통해 사고 맥락 키워드를 더 반영하여 답변을 생성하기 때문

**전반적인 코드 확인 후 느낀점**
- 미니프로젝트 3에서 각각의 RMSE를 분석하고 해당 모델을 가중치로 블렌딩을 수행하는 작업을 했는데, 이와 유사하게 여기서도 가중치 평균을 사용한 것을 확인 가능했음
- 새롭게 LLM이라는 모델을 배우고, 이들이 예측을 수행하는 생성형 AI를 만들고, 해당 평가지표로 이들의 예측 성능을 보완할 수 있는 2가지의 평가지표를 사용함
- 여기선 딥러닝 모델을 구축하여 은닉층, 순전파, 역전파를 사용하였는데 이를 통해 AI 모델을 구현하는 방법과 이를 프론트앤드에 넘기는 전체적인 코드를 확인가능했음
- 자언어 기반 유사도 분석을 통하여 테스트 데이터의 사고 정보와 가장 유사한 학습데이터를 찾는 방식으로 구성되어있고, 여기서는 코사인 유사도와 자카드 유사도를 가중 평균을 통해 최종 유사도를 계산하는 방법을 이용함

-> 전체적인 내용이 본 분석에 집중이 되어있진 않으나 색다른 AI 모델 개발이라는 과정 중에서도 분석이 얼마나 유사도 측정 등에 중요하게 작용하는지를 배울 수 있었음